In [1]:
import requests
from bs4 import BeautifulSoup

In [38]:
list_page_event = requests.get("https://misurainternet.it/valori_statistici/?csrfmiddlewaretoken=PmowMH9VasBwM58EEVdl9W1ARGSIvgCyYTL0uODEECL2GxQn952STNPYUtN6rNqQ&operatore=&regione=&anno=2022&periodo=anno")
site = BeautifulSoup(list_page_event.text, "html.parser")

In [43]:
table = site.find("table", class_="results")

In [44]:
header = [item.text for item in table.find("thead").find_all("th")]
     

In [45]:
header

['Operatore',
 'Profilo',
 'Velocità (down/up)',
 'Territorio',
 'Periodo',
 'Tipo',
 'Media',
 'Dev. Std.',
 'Min',
 'Max',
 'TP%',
 'Misure']

In [52]:
operatore = [item.text for item in table.find_all("td", class_="text-xs-center")]



In [66]:
rows = [row.text.replace(" ","").replace("\n","") for row in table.find_all("tr")]

In [67]:
rows

['OperatoreProfiloVelocità(down/up)TerritorioPeriodoTipoMediaDev.Std.MinMaxTP%Misure',
 'AcanthoBigBand7,0Mbps512.0kbpsNazionaleGen2022Dic2022Download6,8Mbps2,96,8Mbps16907',
 'AcanthoBigBand20,0Mbps1,0MbpsNazionaleGen2022Dic2022Download11,2Mbps1012,310,0Mbps17556',
 'ArubaNone1.000,0Mbps300,0MbpsNazionaleGen2022Dic2022Download788,0Mbps165521,6591,7Mbps25490',
 'BrennercomAllesKlar8,0Mbps384.0kbpsNazionaleGen2022Dic2022Download8,2Mbps263,28,2Mbps11782',
 'BrennercomAllesKlarLight2,0Mbps256.0kbpsNazionaleGen2022Dic2022Download954.6kbps968,70.0kbps11843',
 'BritishTelecomBTADSL7Mega,VipPremium,VipDuet,VipAssolo7,0Mbps832.0kbpsNazionaleGen2022Dic2022Download5,8Mbps1295,63,7Mbps34815',
 'BritishTelecomVipClub7Mega,VipSmart7Mega,VipDuet,VipAssolo7,0Mbps1,0MbpsNazionaleGen2022Dic2022Download4,9Mbps2927,00.0kbps105847',
 'BritishTelecomVipDuet,VipAssolo20,0Mbps1,0MbpsNazionaleGen2022Dic2022Download14,4Mbps2348,911,5Mbps77186',
 'EnelEnergiaFIBRADIMELITA\rDiamondFibra1.000,0Mbps300,0MbpsNazion